In [2]:
import pandas as pd
import numpy as np

# Data Cleaning & Manipulation
With Pandas, we can do more than just select data that is already there. 
We can add new columns to our datasets, apply functions, iterate thorugh each row in the dataframe, and more.

This is where we move from "pandas for exploring our data" to "pandas for getting our data ready to feed into models".

In [3]:
wine = pd.read_csv('data/wine_reviews/winemag-data_first150k.csv', index_col=0)
energy = pd.read_csv('data/energy/PJM_Load_hourly.csv', parse_dates=True, index_col=0)

### Dealing with Null Values:

In [4]:
print('Does the wine df conatin nulls?:', wine.isnull().any().any())
print('Does the energy df contain nulls?:', energy.isnull().any().any())

Does the wine df conatin nulls?: True
Does the energy df contain nulls?: False


In [5]:
wine.isnull().any().any()

True

**Question:** why did we use ".any()" twice when asking this question?

Now, we need to deal with the null values in the wine table.
How we deal with them will depend on the data type of the column in which null values are found, and how we hope to use the data.

If a wine doesn't have a region listed, we can still get a lot of information about it from things like its country. But, if a wine doesn't have a price listed, it's difficult to get much information out of it. 

So, if a wine has a null value for either of its regions, we will set it to be "unknown". But, if it has a null price, we will drop the row.

In [6]:
wine.isnull().any()

country         True
description    False
designation     True
points         False
price           True
province        True
region_1        True
region_2        True
variety        False
winery         False
dtype: bool

In [7]:
print('Shape of wine table before is: ', wine.shape)

null_country = wine[wine['country'].isnull()]
print('Shape of null countries table is:', null_country.shape)

wine = wine[~wine.index.isin(null_country.index)]
print('Shape of wine table after is: ', wine.shape)

Shape of wine table before is:  (150930, 10)
Shape of null countries table is: (5, 10)
Shape of wine table after is:  (150925, 10)


The 5 rows where a NaN value existed for country have been dropped.

Now we'll do the same for rows with a null price.

In [8]:
print('Shape of wine table before is: ', wine.shape)

null_price = wine[wine['price'].isnull()]
print('Shape of null prices table is:', null_price.shape)

wine = wine[~wine.index.isin(null_price.index)]
print('Shape of wine table after is: ', wine.shape)

Shape of wine table before is:  (150925, 10)
Shape of null prices table is: (13695, 10)
Shape of wine table after is:  (137230, 10)


Wow! We lost a lot of rows getting rid of the null price rows.
Can you think of another way we could deal with null prices that doesn't mean losing almost 1/10 of our data? What are the pros and cons to using these other methods?

In [9]:
print('Shape of wine table before is: ', wine.shape)

null_price = wine[wine['price'] == 'unknown']
print('Shape of null prices table is:', null_price.shape)

wine = wine[~wine.index.isin(null_price.index)]
print('Shape of wine table after is: ', wine.shape)

Shape of wine table before is:  (137230, 10)
Shape of null prices table is: (0, 10)
Shape of wine table after is:  (137230, 10)


/anaconda3/lib/python3.6/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


Now, for the other columns in which there are nulls, we can use `df.fillna()` to populate the null values with a value of our choosing. 

This is where we have to make some decisions. Improperly setting variables can have unintended consequences on any analysis we plan to do later using this data.

In [10]:
wine.isnull().any()

country        False
description    False
designation     True
points         False
price          False
province       False
region_1        True
region_2        True
variety        False
winery         False
dtype: bool

The only columns where we still have nulls are regions and designations.
To avoid having to drop out more rows, we will use "fillna()".

In [11]:
wine.fillna('unknown', inplace=True)

We now have no null values in our dataset!

In [12]:
wine.isnull().any()

country        False
description    False
designation    False
points         False
price          False
province       False
region_1       False
region_2       False
variety        False
winery         False
dtype: bool

In [14]:
wine.head()

,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,unknown,Tinta de Toro,Bodega Carmen Rodríguez
2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,unknown,Provence red blend,Domaine de la Bégude


### Creating new columns from existing ones
In pandas, it's easy to make a new column from existing ones. 

In [44]:
wine.loc[:, 'Region'] = wine.loc[:, 'region_1'] + '_' + wine.loc[:, 'region_2']

#### Using a function + apply to make a new column:
Let's say we want to make a new column with an ID we can use to identify the records.
We can use a helper function + the apply method in pandas to apply it to all rows.
Apply looks through each row or column of a dataframe (depending on Axis set).

In [41]:
def add_unique_id(row):
    row['Unique ID'] = (row['country'] + "_" + 
                        row['winery'] + "_" + 
                        str(np.random.randint(low=1, high=1000)))
    return row

In [42]:
wine = wine.apply(add_unique_id, axis=1)

In [43]:
wine.head()

,country,description,designation,points,price,province,region_1,region_2,variety,winery,Unique ID,Region
0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz,US_Heitz_681,Napa Valley_Napa
1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,unknown,Tinta de Toro,Bodega Carmen Rodríguez,Spain_Bodega Carmen Rodríguez_581,Toro_unknown
2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley,US_Macauley_777,Knights Valley_Sonoma
3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi,US_Ponzi_101,Willamette Valley_Willamette Valley
4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,unknown,Provence red blend,Domaine de la Bégude,France_Domaine de la Bégude_341,Bandol_unknown


We can use the same approach to make a column for "great and cheap" wines, defined here as $50 or less and with a score >95.

In [45]:
def helper(row):
    if (row['price'] < 50) & (row['points'] >=95):
        x = 1
    else:
        x = 0
    return x

In [46]:
wine.price = wine.price.astype('int')

In [47]:
wine['great and cheap'] = wine.apply(helper, axis=1)

We can then use that boolean value to sort on the wines.

In [48]:
wine[wine['great and cheap']==1]

,country,description,designation,points,price,province,region_1,region_2,variety,winery,Unique ID,Region,great and cheap
11,US,"From 18-year-old vines, this supple well-balan...",Estate Vineyard Wadensvil Block,95,48,Oregon,Ribbon Ridge,Willamette Valley,Pinot Noir,Patricia Green Cellars,US_Patricia Green Cellars_852,Ribbon Ridge_Willamette Valley,1
12,US,A standout even in this terrific lineup of 201...,Weber Vineyard,95,48,Oregon,Dundee Hills,Willamette Valley,Pinot Noir,Patricia Green Cellars,US_Patricia Green Cellars_116,Dundee Hills_Willamette Valley,1
20,US,Heitz has made this stellar rosé from the rare...,Grignolino,95,24,California,Napa Valley,Napa,Rosé,Heitz,US_Heitz_149,Napa Valley_Napa,1
24,US,"San Jose-based producer Adam Comartin heads 1,...",R-Bar-R Ranch,95,45,California,Santa Cruz Mountains,Central Coast,Pinot Noir,Comartin,US_Comartin_448,Santa Cruz Mountains_Central Coast,1
2439,US,This is possibly the most savory rendition of ...,Estate,96,40,California,San Luis Obispo County,Central Coast,Syrah,Stolo,US_Stolo_99,San Luis Obispo County_Central Coast,1
2735,France,A hint of caramelized apple clings to the nose...,Osterberg Grand Cru,95,38,Alsace,Alsace,unknown,Riesling,Cave de Ribeauvillé,France_Cave de Ribeauvillé_57,Alsace_unknown,1
2737,US,"This splendid, resonant, beautifully detailed ...",Gran Moraine Vineyard,95,45,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Aberrant Cellars,US_Aberrant Cellars_262,Willamette Valley_Willamette Valley,1
2743,France,There is something heady about this: a hint of...,Engelberg Grand Cru,95,45,Alsace,Alsace,unknown,Riesling,Domaine Pfister,France_Domaine Pfister_341,Alsace_unknown,1
2746,France,"After a touch of funky reduction, beautiful hi...",Steingrubler Grand Cru,95,44,Alsace,Alsace,unknown,Riesling,Domaine Barmès-Buecher,France_Domaine Barmès-Buecher_149,Alsace_unknown,1
2755,France,The wine is rich with layers of wood aging and...,unknown,95,35,Bordeaux,Pessac-Léognan,unknown,Bordeaux-style White Blend,Château Latour-Martillac,France_Château Latour-Martillac_794,Pessac-Léognan_unknown,1


**Exercise:** Using a function, make a column that indicates if a wine comes from Africa.

**Exercise:** Make a new column in the energy dataset, showing the energy in time T-1 for each time T. Hint: there is a built-in method in Pandas that can help with this, no complicated functions needed :)

**Exercise:** Re-read the CSV for the wine data, and write a function to take care of all the nan values. Bonus points if it includes a test.